In [197]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.special import erfc
import HammingCode
import Utils

from scipy.special import erfc

# tamanho da amostra
tamanho = 1e5
variancia = 1

# amostra
amostra_avaliada = np.random.choice([0, 1], size=int(tamanho))

Eb_dB_values = np.arange(-2, 11, 1)

In [198]:
def decode_sample(data):
    decoded_data = []
    for i in range(0, len(data) - 6, 7):
        decoded_data.extend(HammingCode.decode(data[i:i + 7]).T)
    return decoded_data

def encode_sample(data):
    encoded_data = []
    for i in range(0, len(data) - 3, 4):
        encoded_data.extend(HammingCode.encode(data[i:i + 4]))
    return encoded_data

In [199]:
def calcular_BER_simulada(amostra, noise_rate):

    amostra_codificada = encode_sample(amostra)
    
    mascara = np.zeros(len(amostra_codificada), dtype=int)
    
    indices_one = np.random.choice(len(amostra_codificada), int(noise_rate * len(amostra_codificada)), replace=False)
    
    #print("Noise rate" )
    #print(noise_rate)
    #print(" indices")
    #print(len(indices_one))#
   
    mascara[indices_one] = 1
   

    for i in range(len(amostra_codificada)):
        if amostra_codificada[i] == 0.0:
            amostra_codificada[i] = 0
        else:
            amostra_codificada[i] = 1

    # mascara = np.random.rand(len(amostra_codificada)) < noise_rate
    potencia_amostra = np.mean(np.array(amostra_codificada)**2)
    
  #  if potencia_amostra == 0:
   #     return None, None
    #potencia_amostra = potencia_amostra if potencia_amostra > 0 else 1
    
    potencia_ruido = np.mean(np.array(mascara)**2) 
    
    if potencia_ruido == 0:
        return None, None
    potencia_ruido = potencia_ruido if potencia_ruido > 0 else 1
    
    amostra_corrompida = np.bitwise_xor(amostra_codificada, mascara)
        
    # Decodificar y como sendo sign(x + N)
    amostra_decodificada = decode_sample(amostra_corrompida)
    
    # Calcular a taxa de erro de bit
    #print(amostra)
    
   # print(amostra_decodificada)
  #  
    ber = np.sum(amostra_decodificada != amostra) / len(amostra)
    #print( np.sum(amostra_decodificada != amostra) )
    return ber, 10*np.log10(potencia_amostra/potencia_ruido)

In [ ]:
BER_simulada = []
SNRs = []
noise_rates = np.linspace(0.0001, 0.1, 10000)
#noise_rates = np.array([0.01])
for noise in noise_rates:
    
        
    ber, snr = calcular_BER_simulada(amostra_avaliada, noise)
    #if (noise == 0.01):
    ##    print(ber)
     #   print(snr)
    SNRs.append(snr)
    BER_simulada.append(ber)

BER_simulada = np.array(BER_simulada)

# Calcular a BER teórica
Eb_values = 10**(Eb_dB_values / 10)
BER_teorica = 0.5 * erfc(np.sqrt(Eb_values))

In [ ]:
''# Plotar as taxas de erro de bit simuladas e teóricas
plt.figure(figsize=(10, 6))
plt.semilogy(SNRs, BER_simulada,  label='BER Simulada')
#plt.semilogy(Eb_dB_values, BER_teorica, label='BER Teórica')
plt.xlabel('SNR (dB)')
plt.ylabel('BER')
plt.grid(True, which='both')
plt.legend()
plt.title('BER Simulada vs BER Teórica')
plt.show()